# DDPM Training on Google Colab

This notebook trains a DDPM model on MNIST using GPU acceleration.


In [ ]:
# Install dependencies
!pip install denoising_diffusion_pytorch torch torchvision matplotlib tqdm

# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# Clone your repository
!git clone https://github.com/yatuzhang/ddpm-mnist.git
%cd ddpm-mnist

# List files to verify
!ls -la


In [ ]:
# Run training with GPU acceleration
!python train_ddp.py --epochs 50 --batch_size 128 --save_freq 5


In [ ]:
# Run the demo to generate samples
!python demo_ddpm.py


In [ ]:
# Download results
from google.colab import files
import zipfile
import os

# Create zip file with results
with zipfile.ZipFile('ddpm_results.zip', 'w') as zipf:
    for root, dirs, files in os.walk('.'):
        for file in files:
            if file.endswith(('.png', '.pth', '.json')):
                zipf.write(os.path.join(root, file))

# Download the zip file
files.download('ddpm_results.zip')
